<a href="https://colab.research.google.com/github/guillaumebrethes/Opply/blob/main/S_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Connection with googlecolab

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


### Importation module



---



* test connection gitub 15:30 to 16:10 : without result - 28 dec
* starting code at 16:15


---



In [ ]:
import numpy as np
%matplotlib inline
import pandas as pd
import seaborn as sns
import calendar
import matplotlib.pyplot as plt

quotes = pd.read_csv("/content/drive/MyDrive/Opply/Opply/d5d9348e5c87368cc1b74e6d4dde9794/Quotes.csv")
request = pd.read_csv('/content/drive/MyDrive/Opply/Opply/d5d9348e5c87368cc1b74e6d4dde9794/Requests.csv')

### Exploration

In [ ]:
print(request.shape)
print(quotes.shape)

(389, 4)
(492, 6)


In [ ]:
request.head()

In [ ]:
quotes.head()

In [ ]:
request.info()

In [ ]:
quotes.info()

* Identified unique values for each column.


In [ ]:
for col in quotes.columns:
  unique_value = quotes[col].unique()
  print(f"> : {col} ==> : {unique_value}")

  value_counts = quotes[col].value_counts(normalize= True).round(2) * 100
  value_counts_with_percent = value_counts.map("{:.2f} %".format)

  print(value_counts_with_percent)
  print(f'- {quotes[col].dtypes} -')
  print('---------------------------')

In [ ]:
for col in request.columns:
  unique_value = request[col].unique()
  print(f"> : {col} ==> : {unique_value}")

  value_counts = request[col].value_counts(normalize= True).round(2) * 100
  value_counts_with_percent = value_counts.map("{:.2f} %".format)

  print(value_counts_with_percent)
  print(f'- {request[col].dtypes} -')
  print('---------------------------')

### Quotes

In [ ]:
quotes["Status"].value_counts()

Expired             290
Available           124
Ordered              58
Declined             17
Contract Created      1
pending_payment       1
Proceeded             1
Name: Status, dtype: int64

* replace nan by 'no' on column "Samples Available" and "Documents Provided"

In [ ]:
quotes["Documents Provided"] = quotes["Documents Provided"].fillna('No')

In [ ]:
quotes["Documents Provided"].isna().sum()

0

In [ ]:
quotes["Samples Available"] = quotes["Samples Available"].fillna('No')

In [ ]:
quotes["Samples Available"].isna().sum()

0



---


* stop coding at 17:06 - 28 dec. (1h)

---
* 29 dec - start coding at 6:15



---



In [ ]:
# Filter the dataframe based on the value of "status" == Ordered

quotes_ordered = quotes[quotes['Status'] == 'Ordered']
print(quotes_ordered.shape)
quotes_ordered.head()


(58, 6)


,Request UUID,Quote Unit Price,Status,Decline Reason,Samples Available,Documents Provided
28,fca03c4f-3f16-40d0-b87c-16d9343e4a52,£4.65,Ordered,NaN,Yes,Yes
30,d596642b-c417-49e6-aa3b-ec6185fb2b9f,£0.58,Ordered,NaN,No,Yes
79,9cf11f62-7b1e-4a7f-9c40-c12ad721a72b,£1.66,Ordered,NaN,No,Yes
80,3c583881-a99d-42ca-a81b-648bc6884976,£1.58,Ordered,NaN,No,Yes
84,2473d2dc-0a5c-4a61-a73d-3050faf0f7a0,£3.77,Ordered,NaN,Yes,No


* 11.79% of quotes are Ordered

In [ ]:
# column : Samples Available = Yes / No in (%)

(quotes_ordered['Samples Available']).value_counts(normalize= True) * 100


No     60.344828
Yes    39.655172
Name: Samples Available, dtype: float64

In [ ]:
# column : Documents Provided = Yes / No in (%)

(quotes_ordered['Documents Provided']).value_counts(normalize= True) * 100

Yes    63.793103
No     36.206897
Name: Documents Provided, dtype: float64

In [ ]:
# column : Samples Available and Documents Provided = "Yes" in the same time

len(quotes_ordered[(quotes_ordered['Samples Available'] == 'Yes') & (quotes_ordered['Documents Provided'] == 'Yes')])


19

* On accepted quotes, approximately:
 * 40% have Samples Available
 * 36% have Documents Provided
 * 33% have both Samples Available and Documents Provided

### Merge quotes_order with Request



* understanding why a quote is accepted based on the
 * Target Unit Price
 * Current Unit Price
 * Quote Unit Price

---



In [ ]:
# inner merge

print(quotes_ordered.shape)
quotes_ordered_request = quotes_ordered.merge(right= request, on= 'Request UUID', how = 'inner')
print(quotes_order_request.shape)
quotes_ordered_request.head()

(58, 6)
(58, 9)


,Request UUID,Quote Unit Price,Status,Decline Reason,Samples Available,Documents Provided,Current Unit Price,Target Unit Price,Supply Priority
0,fca03c4f-3f16-40d0-b87c-16d9343e4a52,£4.65,Ordered,NaN,Yes,Yes,£4.50,£4.50,Price
1,d596642b-c417-49e6-aa3b-ec6185fb2b9f,£0.58,Ordered,NaN,No,Yes,£1.00,£1.00,Price
2,9cf11f62-7b1e-4a7f-9c40-c12ad721a72b,£1.66,Ordered,NaN,No,Yes,£1.66,£1.66,Price
3,3c583881-a99d-42ca-a81b-648bc6884976,£1.58,Ordered,NaN,No,Yes,£1.66,£1.66,Price
4,2473d2dc-0a5c-4a61-a73d-3050faf0f7a0,£3.77,Ordered,NaN,Yes,No,£3.00,£3.00,Price


In [ ]:
# delete £ on columns 'price'

quotes_ordered_request['Quote Unit Price'] = quotes_ordered_request['Quote Unit Price'].replace('£', '', regex=True)
quotes_ordered_request['Current Unit Price'] = quotes_ordered_request['Current Unit Price'].replace('£', '', regex=True)
quotes_ordered_request['Target Unit Price'] = quotes_ordered_request['Target Unit Price'].replace('£', '', regex=True)

In [ ]:
quotes_ordered_request.head()

,Request UUID,Quote Unit Price,Status,Decline Reason,Samples Available,Documents Provided,Current Unit Price,Target Unit Price,Supply Priority
0,fca03c4f-3f16-40d0-b87c-16d9343e4a52,4.65,Ordered,NaN,Yes,Yes,4.50,4.50,Price
1,d596642b-c417-49e6-aa3b-ec6185fb2b9f,0.58,Ordered,NaN,No,Yes,1.00,1.00,Price
2,9cf11f62-7b1e-4a7f-9c40-c12ad721a72b,1.66,Ordered,NaN,No,Yes,1.66,1.66,Price
3,3c583881-a99d-42ca-a81b-648bc6884976,1.58,Ordered,NaN,No,Yes,1.66,1.66,Price
4,2473d2dc-0a5c-4a61-a73d-3050faf0f7a0,3.77,Ordered,NaN,Yes,No,3.00,3.00,Price


In [ ]:
# convert columns 'price' to integer

columns_to_convert = ['Quote Unit Price', 'Current Unit Price', 'Target Unit Price']
quotes_ordered_request[columns_to_convert] = quotes_ordered_request[columns_to_convert].astype(float).round(2)


In [ ]:
quotes_ordered_request.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58 entries, 0 to 57
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Request UUID        58 non-null     object 
 1   Quote Unit Price    58 non-null     float64
 2   Status              58 non-null     object 
 3   Decline Reason      0 non-null      object 
 4   Samples Available   58 non-null     object 
 5   Documents Provided  58 non-null     object 
 6   Current Unit Price  58 non-null     float64
 7   Target Unit Price   58 non-null     float64
 8   Supply Priority     39 non-null     object 
dtypes: float64(3), object(6)
memory usage: 4.5+ KB


In [ ]:
# delete rows with values = 0,01

quotes_ordered_request = quotes_ordered_request.loc[(quotes_ordered_request['Current Unit Price'] != 0.01) & (quotes_ordered_request['Target Unit Price'] != 0.01)]


* Addition of a column **'Price Difference (%)'** that calculates the percentage between the value requested by the buyer **Target Unit Price** and the price provided on the quote by the buyer **Quote Unit Price.**

In [ ]:
quotes_ordered_request['Price Difference (%)'] = ((quotes_ordered_request['Quote Unit Price'] - quotes_ordered_request['Target Unit Price']) / quotes_ordered_request['Target Unit Price']) * 100


<ipython-input-149-6c4dddc26466>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  quotes_ordered_request['Price Difference (%)'] = ((quotes_ordered_request['Quote Unit Price'] - quotes_ordered_request['Target Unit Price']) / quotes_ordered_request['Target Unit Price']) * 100


In [ ]:
quotes_ordered_request.head()

,Request UUID,Quote Unit Price,Status,Decline Reason,Samples Available,Documents Provided,Current Unit Price,Target Unit Price,Supply Priority,Price Difference (%)
0,fca03c4f-3f16-40d0-b87c-16d9343e4a52,4.65,Ordered,NaN,Yes,Yes,4.50,4.50,Price,3.333333
1,d596642b-c417-49e6-aa3b-ec6185fb2b9f,0.58,Ordered,NaN,No,Yes,1.00,1.00,Price,-42.000000
2,9cf11f62-7b1e-4a7f-9c40-c12ad721a72b,1.66,Ordered,NaN,No,Yes,1.66,1.66,Price,0.000000
3,3c583881-a99d-42ca-a81b-648bc6884976,1.58,Ordered,NaN,No,Yes,1.66,1.66,Price,-4.819277
4,2473d2dc-0a5c-4a61-a73d-3050faf0f7a0,3.77,Ordered,NaN,Yes,No,3.00,3.00,Price,25.666667


* We could add 2 more columns for:
 * Price negotiated upwards.
 * Price negotiated downwards.

### Request

In [ ]:
request.head()

,Request UUID,Current Unit Price,Target Unit Price,Supply Priority
0,820412eb-8d4d-4d6c-a5e3-cfc977cd6d48,£3.50,£3.00,Price
1,08f1a913-1318-4656-9418-ce580a9871cb,£0.01,£0.01,Price
2,8cd6fd33-a1bc-43cb-9c63-df7d629d5362,£0.01,£0.01,Price
3,9388fbf6-410b-4f85-999a-1ff4d4cb733b,£0.01,£0.01,Price
4,37d05dfe-7f47-4eed-8e78-9f74c303bcad,£0.01,£0.01,Price


In [ ]:
# delete £
request['Current Unit Price'] = request['Current Unit Price'].replace('£', '', regex=True)
request['Target Unit Price'] = request['Target Unit Price'].replace('£', '', regex=True)

In [ ]:
# delete rows with values = 0,01

request = request.loc[(request['Current Unit Price'] != '0.01') & (request['Target Unit Price'] != '0.01')]


In [ ]:
request.shape

(354, 4)

In [ ]:
# convert columns 'price' to integer

request[columns_to_convert] = request[columns_to_convert].replace({',': ''}, regex=True).astype(float).round(2)

columns_to_convert = ['Current Unit Price', 'Target Unit Price']
request[columns_to_convert] = request[columns_to_convert].astype(float).round(2)


In [ ]:
request.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 354 entries, 0 to 388
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Request UUID        354 non-null    object 
 1   Current Unit Price  354 non-null    float64
 2   Target Unit Price   354 non-null    float64
 3   Supply Priority     258 non-null    object 
dtypes: float64(2), object(2)
memory usage: 13.8+ KB




---



* 29 dec : stop coding at 8:00. (2h)


---

